In [29]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from mpltools import style
from mpltools import layout
import seaborn as sns
style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

In [30]:
train = pd.read_csv("train.csv")
train_copy = train
test = pd.read_csv("test.csv")
daily = pd.read_csv("daily_aggregate.csv")
roaming = pd.read_csv("roaming_monthly.csv")
calendar = pd.read_csv("calendar_ref.csv")
contract = pd.read_csv("contract_ref.csv")

In [75]:
hoba = contract.merge(train)
hoba["COUNT"] = 1
hoba = hoba.groupby(by="HANDSET_NAME").sum()
hoba["Probability"] = hoba["TARGET"] *1.0 / hoba["COUNT"] 
c = hoba.sort(columns=["COUNT"],ascending=False)[["COUNT", "Probability"]]

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,COUNT,Probability
HANDSET_NAME,,
Fibocom Wireless G610,6981,0.000000
Vodafone K4201-Z,3298,0.290782
Apple iPhone 6,2884,0.404993
Apple iPhone 5S,1662,0.362214
Apple iPhone 6 Plus,1408,0.419744
Samsung Galaxy Grand Prime DUOS,1247,0.227747
Samsung N9005 Galaxy Note 3,1166,0.298456
Samsung Galaxy Grand Neo DUOS,1065,0.200000
Quectel Wireless Solutions M95 Series,996,0.002008


In [78]:
hoba = contract.merge(train)
hoba["COUNT"] = 1
hoba = hoba.groupby(by="RATE_PLAN").sum()
hoba["Probability"] = hoba["TARGET"] *1.0 / hoba["COUNT"] 
c = hoba.sort(columns=["Probability"],ascending=False)[["COUNT", "Probability"]]

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [79]:
c

,COUNT,Probability
RATE_PLAN,,
Business Unified USB 3.5 GB,2,1.000000
New Bouquet Ctrl 200,1,1.000000
Business Unified USB 12 GB,1,1.000000
2011 Business Bouquet 500,1,1.000000
Business Bouquet 25 Control,1,1.000000
Business Ass. Engineering,7,0.857143
2011 Bouquet Ctrl 200,4,0.750000
Business USB 100,3,0.666667
New VF Data 250 business,3,0.666667


In [81]:
d = daily.merge(train)

In [87]:
d.groupby("CELL_KEY").mean()

,CONTRACT_KEY,CALL_DATE_KEY,TOTAL_CONSUMPTION,NO_OF_SESSIONS,206_SESSION_COUNT,206_USAGE,207_SESSION_COUNT,207_USAGE,208_SESSION_COUNT,208_USAGE,209_SESSION_COUNT,209_USAGE,210_SESSION_COUNT,210_USAGE,TARGET
CELL_KEY,,,,,,,,,,,,,,,
890,1.498564e+08,6350.575170,7043391.805044,7.847721,1262.601358,4802.621726,1473.441319,5376.122211,1543.366634,6186.987876,1628.411736,5814.816683,1537.456838,5356.713385,0.380213
6342542,1.523263e+08,6353.476190,5306873.350649,10.333333,823.212121,2661.787879,1158.168831,3245.220779,1285.194805,3049.701299,1576.077922,3323.233766,1425.774892,2455.783550,0.285714
6346638,1.433210e+08,6347.000000,3174400.000000,1.000000,334.000000,6445.000000,332.000000,2615.000000,521.000000,4565.000000,431.000000,4941.000000,375.000000,5031.000000,1.000000
6349542,1.480294e+08,6352.660714,28784676.571429,1.250000,352.517857,3125.642857,280.142857,2129.517857,361.625000,2111.053571,351.589286,1839.410714,326.267857,2953.142857,0.285714
6349544,1.493711e+08,6352.478261,25389708.985507,3.173913,564.579710,4553.043478,636.985507,3469.217391,775.695652,6235.101449,633.217391,2823.275362,567.811594,2446.710145,0.739130
6352087,1.485636e+08,6347.416667,60896256.000000,9.300000,818.833333,4366.400000,880.516667,4102.316667,1106.100000,4349.233333,1229.350000,5285.266667,1423.466667,7035.850000,0.633333
6368769,1.523232e+08,6351.335664,5227412.587413,3.174825,388.902098,662.447552,383.377622,654.678322,315.244755,991.398601,317.328671,881.503497,367.342657,590.734266,0.020979
6369124,1.467152e+08,6351.875000,3129600.000000,17.312500,1801.250000,322.125000,1375.250000,320.937500,2540.250000,767.812500,2611.375000,470.812500,3075.750000,537.312500,0.000000
6369126,1.521687e+08,6348.500000,785066.666667,1.166667,482.750000,158.833333,386.916667,182.833333,280.416667,132.083333,282.333333,120.916667,312.083333,175.750000,0.000000
